In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pandas import DataFrame as df
from scipy.stats import zscore
import seaborn as sns
import re
import os

from ydata_profiling import ProfileReport
import sweetviz as sv

/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [1]:
df = pd.read_csv("./data/OnlinePaymentsFraudDetection.csv", na_values=[' ', '_', '_______'], low_memory=False)
df

In [4]:
df.dtypes

step                int64
type               object
amount            float64
nameOrig           object
oldbalanceOrg     float64
newbalanceOrig    float64
nameDest           object
oldbalanceDest    float64
newbalanceDest    float64
isFraud             int64
isFlaggedFraud      int64
dtype: object

In [5]:
# Criar um dicionário para armazenar o número de valores únicos por coluna
num_valores_unicos = {}

# Iterar sobre cada coluna e contar os valores únicos
for coluna in df.columns:
    num_valores_unicos[coluna] = df[coluna].nunique()

# Converter o dicionário em uma Series para facilitar a visualização
serie_num_valores_unicos = pd.Series(num_valores_unicos)

# Exibir o número de valores únicos por coluna
print("Número de valores únicos por coluna:")
serie_num_valores_unicos

Número de valores únicos por coluna:


step                  743
type                    5
amount            5316900
nameOrig          6353307
oldbalanceOrg     1845844
newbalanceOrig    2682586
nameDest          2722362
oldbalanceDest    3614697
newbalanceDest    3555499
isFraud                 2
isFlaggedFraud          2
dtype: int64

In [3]:
valores_vazios_por_coluna = df.isna().sum()

valores_vazios_por_coluna

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64